In [1]:
from lib.fxstreet_calendar  import FxstreetCalendar
from lib.vicki_operator import Operator

In [2]:
api = FxstreetCalendar()

In [3]:
op = Operator('Economic Calendar Definitions')

In [5]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
from datetime import date
import time
import requests
import json

In [6]:
from_ = dt.today()+timedelta(days=1)
to_   = dt.today()+timedelta(days=7)
events_master = api.get_calendar_df(from_=from_, to_=to_)
for x in range(1, 25):
    from_ = dt.today()+timedelta(days=x*7 + 1)
    to_ = dt.today()+timedelta(days=(x+1)*7)
    events_master = pd.concat([events_master, api.get_calendar_df(from_=from_, to_=to_)])
    time.sleep(3)

In [8]:
high_volatility = events_master[events_master['volatility']=='HIGH']

In [9]:
top_names = high_volatility.value_counts('name')
top_names = list(top_names[top_names>1].index)

In [10]:
high_volatility = high_volatility[high_volatility['name'].isin(top_names)]

In [11]:
mid_volatility = events_master[events_master['volatility']=='MEDIUM']

In [12]:
top_names = mid_volatility.value_counts('name')
top_names = list(top_names[top_names>3].index)

In [13]:
mid_volatility = mid_volatility[mid_volatility['name'].isin(top_names)]

In [14]:
out_master = pd.concat([high_volatility, mid_volatility])
out_master = out_master.drop_duplicates(['name'])

In [15]:
out_master['translated_name'] = out_master['name']

In [16]:
len(out_master)

142

In [17]:
known_good_defs = op.workbook.sheet_to_df(sheet='English')
out_master = out_master[~out_master['name'].isin(known_good_defs['name'])]
out_master = pd.concat([known_good_defs, out_master])

### Sim Functions

In [9]:
def select_events(events_df: pd.DataFrame, currencies, max_number):
    events_by_currency = []
    for i in range(5):
        for x in currencies:
            currency_events = events_df[events_df['currencyCode']==x]
            if len(currency_events) > 0:
                if len(currency_events) <= i:
                    events_by_currency.append(currency_events)
                else:
                    events_by_currency.append(currency_events.sort_values('volatility').head(i))
        if sum(len(y) for y in events_by_currency) >= max_number:
            break
        elif i < 4:
            events_by_currency = []
    selected_rows = pd.concat(events_by_currency)
    if len(selected_rows) > max_number:
        return selected_rows.sample(max_number)
    return selected_rows

### Approach
Take high and medium volatility events that appear often

Calculate approximately minimum cover for the "holes" left

In [26]:
out_master = op.workbook.sheet_to_df(sheet='Economic Calendar Definitions')
print(len(out_master))
# l1_currencies = ['USD', 'CNY', 'JPY', 'KRW', 'AUD']
l1_currencies = ['USD', 'EUR', 'JPY', 'GBP', 'AUD', 'NZD', 'KRW', 'HKD', 'SGD', 'CNY', 'CAD']

202


In [27]:
countries = set([])
patchers = []
from_ = date.today() + timedelta(days=150)
for x in range(148):
    from_ -= timedelta(days=1)
    if from_.weekday() in [5,6]:
        continue
    day_events = events_master[events_master['dateUtc'].dt.date==from_]
    known_coming = pd.merge(left=day_events, right=out_master[['name', 'translated_name']], how='inner', on='name')
    try:
        selected_events = select_events(known_coming, l1_currencies, 6)
        for x in selected_events['countryCode'].unique():
            countries.add(x)
        if len(selected_events) < 6:
            print(from_)
            patchers += day_events.to_dict('records')
    except ValueError as e:
        print(e)
        print(len(known_coming))
len(patchers)

0

In [11]:
active_countries = [
    'US',
    'EMU',
    'JP',
    'AU',
    'NZ',
    'CN',
    'CA',
    'SG',
    'HK',
    'KR',
    'DE',
    'IT',
    'UK',
    'ES',
    'FR'
]
for x in countries:
    if not x in active_countries:
        print(x)

PT
GR
NL
BE
AT
IE
FI


In [28]:
patchers_df['countryCode'].unique()

array(['FR', 'ES', 'US', 'KR', 'PL', 'DE', 'CN', 'ZA', 'RU', 'UK', 'NZ',
       'NL', 'CZ', 'BE', 'AU', 'CH', 'JP'], dtype=object)

In [12]:
patchers_df = pd.DataFrame(patchers).drop_duplicates(['name'])
patchers_df = patchers_df[~patchers_df['name'].isin(out_master['name'])]

In [18]:
len(patchers_df[patchers_df['currencyCode'].isin(l1_currencies)])

24

In [17]:
len(out_master)

173

In [14]:
patchers_df['translated_name'] = patchers_df['name']
patchers_df['definition'] = patchers_df.apply(lambda x: api.get_definition(x['id']), axis=1)

In [19]:
out_master = pd.concat([patchers_df[patchers_df['currencyCode'].isin(l1_currencies)][['name', 'translated_name', 'definition']], out_master])

In [22]:
patchers_df[~patchers_df['name'].isin(out_master['name'])].head(20)

,id,name,dateUtc,previous,unit,currencyCode,countryCode,volatility,potency
1,42c939ba-18f0-40e8-8f7b-c76f7908841a,German Buba Monthly Report,2021-07-19 10:00:00,---,,EUR,DE,LOW,None
5,c6d83305-0483-46a2-b329-74fe52ad5f27,New Loans,2021-07-12 07:00:00,---B,,CNY,CN,LOW,B
6,17099714-2034-4d30-ba2f-45d557069e4a,M2 Money Supply,2021-07-12 07:00:00,---%,%,CNY,CN,LOW,ZERO
7,e8780ab9-7af1-4094-9e58-adcd04068a10,USDA WASDE Report,2021-07-12 16:00:00,---,,USD,US,LOW,None
10,70aaa54a-89a9-4ea9-b1d2-e486e1b0487a,Producer Prices,2021-06-28 06:45:00,---%,%,EUR,FR,LOW,ZERO
11,58907ade-eb22-4931-a1c4-7a5295f60ac9,Services Sentiment,2021-06-28 09:00:00,---,,EUR,EMU,LOW,ZERO
14,f9181763-a205-4b77-9e5a-f6373fc2b4a5,Economic Sentiment Indicator,2021-06-28 09:00:00,---,,EUR,EMU,LOW,ZERO
16,f2cc7f9d-1e93-44b4-a3e8-62d901ea1a5f,Dallas Fed Manufacturing Business Index,2021-06-28 14:30:00,---,,USD,US,LOW,ZERO
18,3cb81986-7ca2-4f0a-ac9a-fbf345cddb99,Credit Card Spending,2021-06-21 03:00:00,---%,%,NZD,NZ,LOW,None
24,829977b9-34e4-413c-a5fb-277092a2fd07,Westpac Consumer Survey,2021-06-14 21:00:00,---,,NZD,NZ,MEDIUM,ZERO


In [23]:
out_master=pd.concat([out_master, patchers_df]).drop_duplicates(['name'])

In [29]:
out_master = out_master[~out_master['name'].isin(['REINZ House Price Index'])]

In [25]:
op.workbook.df_to_sheet(out_master[['name', 'translated_name', 'definition']], sheet='Economic Calendar Definitions')

In [24]:
out_master.tail(10)

,name,translated_name,definition
171,RBA Interest Rate Decision,RBA Interest Rate Decision,The RBA Interest Rate Decision is announced by...
172,RBA Rate Statement,RBA Rate Statement,Decisions regarding the interest rates are mad...
173,AiG Performance of Construction Index,AiG Performance of Construction Index,The AiG Performance of Construction Index is a...
175,Monetary Policy Summary,Monetary Policy Summary,The Monetary Policy Summary contains the outco...
176,Unit Labor Costs,Unit Labor Costs,The Unit Labor Cost shows the total cost of em...
177,RBA Meeting Minutes,RBA Meeting Minutes,The minutes of the Reserve Bank of Australia m...
178,BSI Large Manufacturing Conditions Index,BSI Large Manufacturing Conditions Index,The Business Survey Index (BSI) Large Manufact...
179,Labor Cost,Labor Cost,The quarterly Labor Cost shows the short-term ...
180,Consumer Inflation Expectations,Consumer Inflation Expectations,The Consumer Inflation Expectation presents th...
181,Wholesale Price Index,Wholesale Price Index,The Wholesale Price Index shows the value of s...


### Get ids from events_master, push to google sheets

In [26]:
out_master = out_master[~out_master['name'].isin(known_good_defs['name'])]
print(len(out_master))

57


In [27]:
out_master['definition'] = out_master.apply(lambda x: api.get_definition(x['id']), axis=1)

In [28]:
out_master['translated_name'] = out_master['name']
out_master = pd.concat([known_good_defs, out_master])
op.workbook.df_to_sheet(out_master[['name', 'translated_name', 'definition']], sheet='Economic Calendar Definitions')

In [31]:
len(out_master)

230

In [24]:
op.workbook.df_to_sheet(out_master[['name', 'translated_name', 'definition']], sheet='Economic Calendar Definitions')

### German

In [28]:
DEEPL_AUTH = '128b4f74-108b-c09d-ba5b-a66ae4237dec'
DEEPL_URL = 'https://api.deepl.com/v2/translate'
def get_translation(to_translate, from_lang='EN', to_lang='DE'):
    time.sleep(0.1)
    post_fields = {
        'auth_key': DEEPL_AUTH,
        'text': to_translate,
        'target_lang': to_lang,
        'source_lang': from_lang,
    }
    request = requests.post(DEEPL_URL, params=post_fields)
    response = json.loads(request.content.decode('utf-8'))
    return response['translations'][0]['text']

In [30]:
# german_defs = op.workbook.sheet_to_df(sheet='English')
# german_defs['translated_name'] = german_defs['translated_name'].apply(get_translation)
# german_defs['definition'] = german_defs['definition'].apply(get_translation)
op.workbook.df_to_sheet(german_defs, sheet='German')